# 1. Connect to database

Creating connection to Microsoft SQL Server and two databases with positive and negative sets. We should use generators for batch processing.

In [488]:
import pyodbc
import numpy as np

from pprint import pprint

batch_size = 10

In [489]:
def data_generator(constring, query):
    cnxn = pyodbc.connect(constring)
    cursor = cnxn.cursor()
    cursor.execute(query)
    
    y, t = [], []
    for row in cursor:
        r_text = row.ttext
        r_type = row.ttype
        y.append(r_text)
        t.append(r_type)
        
        if len(y) == batch_size:
            npx = np.array(y)
            npy = np.array(t)
            yield npx, npy
            y, t = [], []
    pyodbc.Connection.close(cnxn)

# 2. Tokenization
We should encode words as their indexes (computed by overall frequency in the dataset).
Using russian http://www.ruscorpora.ru/en/
#### Step 1. Clear dataset. 
#### Step 2. Select meaningful words.
#### Step 3. Calculate frequency of each word
#### Step 4. Replace words by indexes

In [490]:
import nltk
import string
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

hash_map = {}
max_features = 20000

#### Step 1. Clear dataset. 
Select words one by one. Symbols are meaningful because of smiles and emotions

In [491]:
def tokenize(file_text):
    if file_text is not None:
        #firstly let's apply nltk tokenization
        tokens = nltk.word_tokenize(file_text)

        #let's delete punctuation symbols
        stop_words = ([',','\\','/','*','','-','http',';',':','@'])
        tokens = [i for i in tokens if ( i not in stop_words )]

        #deleting stop_words
        #stop_words = stopwords.words('russian')
        #stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'в', '—', 'к', 'на', 'http'])
       
       # tokens = [i for i in tokens if ( i not in stop_words )]

        #cleaning words
        #tokens = [i.replace("«", "").replace("»", "") for i in tokens]

        return tokens
    else:
        return None

We will create a HashMap by using a Python dictionary to store the word frequencies of a book.
A dictionary is an associative array (also known as hashes).
Any key of the dictionary is associated, or mapped, to a value.
The values of a dictionary can be any Python data type, so dictionaries are unordered key-value-pairs.

By creating the dictionary, we will store the words as the keys and the value will represent the count. By doing this, we can retrieve any word without having to recount every single word.

In [492]:

def map_words(tokens):
    
    if tokens is not None:
        for element in tokens:
            # Remove Punctuation
            word = element.replace(",","")
            word = word.replace(".","")
            word = word.lower()
            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None
    

In [493]:
cnstr_positive = 'Trusted_Connection=yes;DRIVER={SQL Server};SERVER=DESKTOP-1RHDOBR\GORDASQL;DATABASE=positive;UID=pyuser;PWD=pypypy'

cnstr_negative = 'Trusted_Connection=yes;DRIVER={SQL Server};SERVER=DESKTOP-1RHDOBR\GORDASQL;DATABASE=negative;UID=pyuser;PWD=pypypy'

In [497]:
def fill_frequency():
    hash_map.clear()
    
    query_positive = "SELECT [ttext], [ttype] FROM [dbo].[sortpos]"
    pdg = data_generator(cnstr_positive, query_positive)
    
    query_negative = "SELECT [ttext], [ttype] FROM [dbo].[sortneg]"
    ndg = data_generator(cnstr_negative, query_negative)
    
    for current_positive_set in pdg:
        for sentence in current_positive_set[0]:
            words = tokenize(sentence)
            map = map_words(words)
            
    for current_negative_set in ndg:
        for sentence in current_negative_set[0]:
            words = tokenize(sentence)
            map = map_words(words)
    
    min_frequency = 2#max(map.values()) - max_features if max(map.values()) > max_features else 2
    map = {key: value for key, value in map.items() if value > min_frequency}
    return map

In [498]:
map = fill_frequency()

Let's check few words

In [499]:
word_list = ['привет',',','дела',')','(']

for word in word_list:
    print('Word: [' + word + '] Frequency: ' + str(map.get(word,0)))

Word: [привет] Frequency: 974
Word: [,] Frequency: 0
Word: [дела] Frequency: 571
Word: [)] Frequency: 151358
Word: [(] Frequency: 181951


In [500]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

maxlen = 60  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [509]:
def train_model():
    max_frequency = max(map.values())
    
    train_cnt = 500
    #cnxn = pyodbc.connect(cnstr_positive)
    #query = "SELECT count(*) as cnt FROM [dbo].[mixedmessages]"
    #cursor = cnxn.cursor()
    #cursor.execute(query)
    #for row in cursor:
    #    train_cnt = row.cnt/2
    #pyodbc.Connection.close(cnxn)
    
    x_train, y_train, x_test, y_test = ([] for i in range(4))
    
    query = "SELECT top 1000 [ttext], [ttype] FROM [dbo].[mixedmessages] order by newid()"
    mdg = data_generator(cnstr_positive, query)
    
    for current_set in mdg:
        for sentence in current_set[0]:
            words = tokenize(sentence)
            w = []
            #print(words)
            for word in words:
                #print(word, map.get(word,0))
                w.append(max_frequency-map.get(word,0))
            if len(x_train)<train_cnt:
                x_train.append(w)
            else:
                x_test.append(w)
                
        for value in current_set[1]:
            if len(y_train)<train_cnt:
                y_train.append(value)
            else:
                y_test.append(value)
                
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    #print(x_train)
    return x_train, y_train, x_test, y_test

In [510]:
x_train, y_train, x_test, y_test = train_model()
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [511]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Build model...


In [512]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))


Train...
Train on 500 samples, validate on 492 samples
Epoch 1/15
500/500 [==============================] - 6s - loss: 0.6552 - acc: 0.7260 - val_loss: 0.6233 - val_acc: 0.6606
Epoch 2/15
500/500 [==============================] - 3s - loss: 0.5646 - acc: 0.8400 - val_loss: 0.5445 - val_acc: 0.7256
Epoch 3/15
500/500 [==============================] - 3s - loss: 0.4479 - acc: 0.8540 - val_loss: 0.3942 - val_acc: 0.8841
Epoch 4/15
500/500 [==============================] - 3s - loss: 0.3348 - acc: 0.8960 - val_loss: 0.2940 - val_acc: 0.9146
Epoch 5/15
500/500 [==============================] - 3s - loss: 0.2423 - acc: 0.9200 - val_loss: 0.2359 - val_acc: 0.9207
Epoch 6/15
500/500 [==============================] - 3s - loss: 0.1953 - acc: 0.9280 - val_loss: 0.2082 - val_acc: 0.9289
Epoch 7/15
500/500 [==============================] - 3s - loss: 0.1949 - acc: 0.9380 - val_loss: 0.2157 - val_acc: 0.9431
Epoch 8/15
500/500 [==============================] - 3s - loss: 0.1722 - acc: 0.944

In [513]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

492/492 [==============================] - 0s     
Test score: 0.140795229169
Test accuracy: 0.95731707414


In [516]:
my_example = np.array([[23, 75, 43, 225, 322]])
my_example = sequence.pad_sequences(my_example, maxlen=maxlen)

In [517]:
model.predict_classes(my_example)

1/1 [==============================] - 0s


array([[1]])

In [518]:
model.predict(my_example)

array([[ 0.53944892]], dtype=float32)

In [212]:
#import pymorphy2
#morph = pymorphy2.MorphAnalyzer()